In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from cmap import ncl_colormap 
from mpl_toolkits.basemap import Basemap, addcyclic
from my_tools import file_dic, plot_dic
from glob import glob

%matplotlib qt

In [2]:
path = '/media/onno/Algemeen/Thesis/'

In [3]:
#load RWP property files
rean_env_gen = xr.open_dataset(path +file_dic['envelope']['ERA5']).sel(latitude=slice(90,0)).squeeze()
rean_v300_gen = xr.open_dataset(path + file_dic['v_wind']['ERA5']).sel(latitude=slice(90,0)).squeeze()
rean_env_gen =  rean_env_gen.assign_coords(longitude=(((rean_env_gen.longitude + 180) % 360) - 180)).sortby('longitude')
rean_v300_gen =  rean_v300_gen.assign_coords(longitude=(((rean_v300_gen.longitude + 180) % 360) - 180)).sortby('longitude')

rean_env_gen=rean_env_gen.rename({'longitude':'lon'});rean_env_gen=rean_env_gen.rename({'latitude':'lat'})
rean_v300_gen=rean_v300_gen.rename({'longitude':'lon'});rean_v300_gen=rean_v300_gen.rename({'latitude':'lat'})
# rean_env_gen = rean_env_gen.sel(time=[bool(i) for i in rean_env_gen.time.dt.hour%24==0])
# rean_v300_gen = rean_v300_gen.sel(time=[bool(i) for i in rean_v300_gen.time.dt.hour%24==0])

ERA5RF_env_gen = xr.open_dataset(path+file_dic['envelope']['ERA5RF'],decode_times=False).squeeze()
GFS_env_gen = xr.open_dataset(path+file_dic['envelope']['GFS'],decode_times=False).squeeze()
# ERA5RF_v300_gen = xr.open_dataset(path+'era5rf_v300_wledit2000-10000_latavg_0-240h_12hourly_2x2nh_jan79-dec19.nc',decode_times=False).squeeze()
# GFS_v300_gen = xr.open_dataset(path+'gefsrf2_v300_control0-252h_6hourly_2x2_dec84-nov19.nc',decode_times=False).squeeze()

GFS_init_time = pd.Timestamp('1800-01-01')
ERA5RF_init_time = pd.Timestamp('1900-01-01')
GFS_env_gen['time']=[pd.Timedelta(i,'hours')+GFS_init_time for i in GFS_env_gen.time.values]
# GFS_v300_gen['time']=[pd.Timedelta(i,'hours')+GFS_init_time for i in GFS_v300_gen.time.values]
ERA5RF_env_gen['time']=[pd.Timedelta(i,'hours')+ERA5RF_init_time for i in ERA5RF_env_gen.time.values]
# ERA5RF_v300_gen['time']=[pd.Timedelta(i,'hours')+ERA5RF_init_time for i in ERA5RF_v300_gen.time.values]

# GFS_env_gen = GFS_env_gen.rename({'lon':'longitude','lat':'latitude'}); GFS_v300_gen = GFS_v300_gen.rename({'lon':'longitude','lat':'latitude'})
# ERA5RF_env_gen = ERA5RF_env_gen.rename({'lon':'longitude','lat':'latitude'}); ERA5RF_v300_gen = ERA5RF_v300_gen.rename({'lon':'longitude','lat':'latitude'})
GFS_env_gen = GFS_env_gen.assign_coords(lon=(((GFS_env_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
# GFS_v300_gen = GFS_v300_gen.assign_coords(lon=(((GFS_v300_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
ERA5RF_env_gen = ERA5RF_env_gen.assign_coords(lon=(((ERA5RF_env_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
# ERA5RF_v300_gen = ERA5RF_v300_gen.assign_coords(lon=(((ERA5RF_v300_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
# GFS_env_gen = GFS_env_gen.sel(lead=GFS_env_gen.lead.values[[bool(i) for i in GFS_env_gen.lead%24==0]])
# GFS_v300_gen = GFS_v300_gen.sel(lead=GFS_v300_gen.lead.values[[bool(i) for i in GFS_v300_gen.lead%24==0]])
# ERA5RF_env_gen = ERA5RF_env_gen.sel(lead=ERA5RF_env_gen.lead.values[[bool(i) for i in ERA5RF_env_gen.lead%24==0]])
# # ERA5RF_v300_gen = ERA5RF_v300_gen.sel(lead=ERA5RF_v300_gen.lead.values[[bool(i) for i in ERA5RF_v300_gen.lead%24==0]])

# GFS_env_gen['lead'] = GFS_env_gen.lead.values//24
# GFS_v300_gen['lead'] = GFS_v300_gen.lead.values//24
# ERA5RF_env_gen['lead'] = ERA5RF_env_gen.lead.values//24
# # ERA5RF_v300_gen['lead'] = ERA5RF_v300_gen.lead.values//24

# GFS_env_gen = GFS_env_gen.rename({'latitude':'lat','longitude':'lon'})
# GFS_v300_gen = GFS_v300_gen.rename({'latitude':'lat','longitude':'lon'})
# ERA5RF_env_gen = ERA5RF_env_gen.rename({'latitude':'lat','longitude':'lon'})
# # ERA5RF_v300_gen = ERA5RF_v300_gen.rename({'latitude':'lat','longitude':'lon'})





In [25]:
coordinatez = [
(54,46,6,14),#Germany
# (44,36,352,360), #Spain
# (54,46,26,34), #Ukraine
(58,50,352,360), #UK
# (42,34,28,36), #Turkey
# (68,60,22,30), #Finland
(66,58,6,14)] #Norway/Sweden
# (60,52,46,54)] #Russia Samara/Kazan region
path = '/media/onno/Algemeen/Thesis/fcst_composite/hovmoller/reanalysis/'
file = 'rawdata/{}_forecast_{}_hovmoller_{}_lon_{}_{}_lat_{}_{}_{}_{}.nc'
eventz = ['persistent_hw','persistent_cw']
event_titlez = ['Persistent Heatwave','Persistent Coldwave']
# fcstz_env = [GFS_env_gen]
# fcstz_v300 = [GFS_v300_gen]
# modelz = ['GFS','ERA5RF']
seasonz = ['JJA','DJF']
modelz = ['GFS','ERA5RF']
model_labelz = ['GEFSRF','ERA5RF']
contour_dic = {'DJF':{'persistent_hw':np.arange(20,35,1),
                     'persistent_cw': np.arange(20,35,2),
                     'short_hw':np.arange(20,35,1),
                     'short_cw':np.arange(20,35,1)},
               'JJA':{'persistent_hw':np.arange(15,25,1),
                     'persistent_cw': np.arange(15,25,1),
                     'short_hw':np.arange(15,25,1),
                     'short_cw':np.arange(15,25,1)}}
lead_day = 5

letterz_ax1 = ['a','b']
letterz_ax2 = ['c','d']
ds_good_event_modelz = []
ds_bad_event_modelz = []
for model in modelz:
    ds_good_eventz = []
    ds_bad_eventz = []
    for i,event in enumerate(eventz):
        ds_good_seasonz = []
        ds_bad_seasonz = []
        for season in seasonz:
            ds_good_regionz = []
            ds_bad_regionz = []
            for lat_0,lat_1,lon_0,lon_1 in coordinatez:
                file_good_env = file.format('good','envelope',event,lon_0,lon_1,lat_0,lat_1,season,model)
                file_bad_env = file.format('bad','envelope',event,lon_0,lon_1,lat_0,lat_1,season,model)
                ds_good = xr.open_dataset(path+file_good_env)
                ds_bad = xr.open_dataset(path+file_bad_env)
                lon_0 = ds_good.attrs['lon_0']
                ds_good = ds_good.assign_coords(lon=(((ds_good.lon-lon_0+180)%360)-180)).sortby('lon')
                ds_bad = ds_bad.assign_coords(lon=(((ds_bad.lon-lon_0+180)%360)-180)).sortby('lon')
                ds_good_regionz.append(ds_good)
                ds_bad_regionz.append(ds_bad)
            ds_good_region = xr.concat(ds_good_regionz,dim='region').mean(dim='region')
            ds_bad_region = xr.concat(ds_bad_regionz,dim='region').mean(dim='region')
            ds_good_seasonz.append(ds_good_region)
            ds_bad_seasonz.append(ds_bad_region)
        ds_good_season = xr.concat(ds_good_seasonz,dim='season').mean(dim='season')
        ds_bad_season = xr.concat(ds_bad_seasonz,dim='season').mean(dim='season')
        ds_good_eventz.append(ds_good_season)
        ds_bad_eventz.append(ds_bad_season)
    ds_good_event = xr.concat(ds_good_eventz,dim='event').mean(dim='event')
    ds_bad_event = xr.concat(ds_bad_eventz,dim='event').mean(dim='event')
    ds_good_event.to_netcdf(path+'rawdata/good_forecast_envelope_hovmoller_persistent_NW_EUROPE_COMBINED_{}.nc'.format(model))
    ds_good_event.to_netcdf(path+'rawdata/bad_forecast_envelope_hovmoller_persistent_NW_EUROPE_COMBINED_{}.nc'.format(model))
    ds_good_event_modelz.append(ds_good_event)
    ds_bad_event_modelz.append(ds_bad_event)



fig,axz=plt.subplots(2,2,figsize=(10,8))
for i in np.arange(len(modelz)):
    ax1 =axz[0,i];ax2=axz[1,i]
    lonz = ds_good_event.lon.values
    dayz = ds_good_event.time.values

    clr_levels_contourf = np.arange(10,25.01,.1)
    clr_levels_contour = np.arange(10,25.01)
    im = ax1.contourf(lonz,dayz,ds_good_event_modelz[i].v.values,cmap=ncl_colormap(),levels=clr_levels_contourf,extend='both')
    cs = ax1.contour(lonz,dayz,ds_good_event_modelz[i].v.values,colors='k',levels=clr_levels_contour,extend='both',linewidths=1)
    ax1.clabel(cs,inline=True,fmt='%.0f')

    ax1.set_title(model_labelz[i] + '\n Good Forecasts',fontsize=15)
    ax1.axvline(x=0,color='grey');ax1.axvline(x=0,color='grey')
    ax1.axhline(y=0,color='grey')
    ax1.set_ylabel('Days before/after onset event',fontsize=12)
    ax1.set_xlabel('Pseudo Longitude [°]',fontsize=12)
    ax1.set_xticks([-120,-60,0,60,120])
    ax1.set_xticklabels([])
    ax1.tick_params(labelsize=12)
    ax1.set_yticks([-10,-5,0,5,10])
    ax1.text(x=0,y=1.1,s=letterz_ax1[i]+')',transform=ax1.transAxes,fontweight='bold',fontsize=17,
                   verticalalignment='top')


    ax2.contourf(lonz,dayz,ds_bad_event_modelz[i].v.values,cmap=ncl_colormap(),levels=clr_levels_contourf,extend='both')
    cs = ax2.contour(lonz,dayz,ds_bad_event_modelz[i].v.values,colors='k',levels=clr_levels_contour,extend='both',linewidths=1)
    ax2.clabel(cs,inline=True,fmt='%.0f')

    ax2.set_title('Bad Forecasts',fontsize=15)
    ax2.axvline(x=0,color='grey');ax2.axvline(x=0,color='grey')
    ax2.axhline(y=0,color='grey')
    ax2.set_ylabel('Days before/after onset event',fontsize=12)
    ax2.set_xlabel('Pseudo Longitude [°]',fontsize=12)
    ax2.set_xticks([-120,-60,0,60,120])
    ax2.set_yticks([-10,-5,0,5,10])
    ax2.tick_params(labelsize=12)
    ax2.text(x=0,y=1.1,s=letterz_ax2[i]+')',transform=ax2.transAxes,fontweight='bold',fontsize=17,
                   verticalalignment='top')

    ax1.label_outer(); ax2.label_outer()
cbar_ax = fig.add_axes([0.2, 0.07, 0.6, 0.02])
cbar = fig.colorbar(im, cax=cbar_ax,orientation='horizontal')
cbar.ax.get_xaxis().set_ticks([10,15,20,25])
cbar.ax.set_xticklabels([10,15,20,25])
cbar.ax.set_xlabel('E (m/s)',fontsize=12)
cbar.ax.tick_params(labelsize=12)

fig.subplots_adjust(left=0.1,right=0.97,bottom=0.16,top=0.92,hspace=0.15,wspace=0.05)

fig.savefig(path+'combined/persistent_extremes_NW_EUROPE.png')

#                 fig.savefig(path+'good_forecast_envelope_hovmoller_{}_COMBINED_{}_{}_alt.png'.format(event,season,model))
#                 plt.close(fig)